# Llama summarizer only applied to top 20 for each user
#### like 20 sec per query so use sparingly 

In [2]:
import pandas as pd
import swifter
from langdetect import detect
from llama_cpp import Llama
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors

c:\Users\ethan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# abstract to vec run on every paper that has a click/like/dislike
#### 15 ms per so not too bad but still kinda slow

In [5]:

nlp = spacy.load("en_core_web_sm")
path = r'llama\word2vec-google-news-300.bin'
model = KeyedVectors.load_word2vec_format(path, binary=True)
def sentenceSimplifier(txt): 
    doc = nlp(txt)
    out = ""
    for token in doc:
        if token.pos_ in ['PROPN', 'VERB','NUM', 'NOUN']:
            out+=token.lemma_+" "
    return out[:-1]
def sentence2vec(txt):
    simplified_txt = sentenceSimplifier(txt)
    words = simplified_txt.split(" ")
    vecs_arr = np.array([model[word] for word in words if word in model]).sum(axis=0)/len(words)
    return  vecs_arr
def reset_recommendations():
    if input('Are you really sure??? If so type "Yes, I understand"') == "Yes, I understand":
        df = pd.DataFrame(columns=['paper_id', 'abstract_vec'])
        #df.to_pickle('user_pref.pkl')
        print('you have reset the recommendation database')
        return df
    else:
        print('You exited the reset')
def get_similarity(old, new):#input is output of sentence2vec()for each abstract
    return cosine_similarity([new], [old])[0][0]
def get_score(new, user_id, pref_df):
    user_pref_df = pref_df[pref_df[user_id+'_weight']!=0]
    user_pref_df['score'] = user_pref_df['abstract_vec'].apply(get_similarity, args=[new,])
    user_pref_df['score'] = user_pref_df['score']*user_pref_df[user_id+'_weight']
    return user_pref_df['score'].sum()

In [6]:
abstract = 'From scary stories to horror films and haunted houses, the horror genre is wildly popular. Although horror aims to elicit fear and anxiety, many people with anxiety are horror fans and some report using horror to cope with anxiety. In this article, we provide a theoretical rationale for why people with anxiety might choose to access and find relief in horror films. First, we discuss aspects of horror that could make it alluring to people with anxiety and how the use of horror may be negatively reinforcing. Next, we examine how engagement with horror could be used to build skills for resilience in generalized situations. We build on processes from evidence-based therapies (i.e., cognitive behavioral therapy and exposure therapy) to explain how horror media has the potential to be used as a therapeutic tool. Finally, we discuss steps for future research on horror as a therapeutic tool for anxiety-related disorders.'
sentence2vec(abstract)

array([ 0.07619389,  0.02606986,  0.00070626,  0.10004665, -0.09712175,
        0.05676771,  0.06184474, -0.08232509,  0.1423718 ,  0.02442899,
       -0.04236145, -0.12389548,  0.03935721,  0.09261126, -0.06630075,
        0.08609562, -0.00209931,  0.13998452, -0.04351534, -0.13364607,
        0.07832729,  0.04329344,  0.01654053,  0.05512085,  0.02173582,
       -0.01960536, -0.03000532,  0.05455235,  0.02008062, -0.07204481,
       -0.03109698, -0.04705636, -0.07829273, -0.03755275, -0.06861147,
       -0.03009208,  0.08954702,  0.01853899,  0.06785452,  0.0249321 ,
        0.05934999, -0.03203223,  0.02805732,  0.0065264 , -0.09514683,
       -0.05722852,  0.00163269,  0.01378098, -0.02443608,  0.07273069,
       -0.07266366,  0.00892214, -0.09674661, -0.02455017,  0.05519144,
       -0.08384312, -0.09266881, -0.06292202,  0.01115058, -0.06674107,
        0.00047259,  0.03584856, -0.00632281,  0.0095023 , -0.01753104,
       -0.02282715, -0.0819288 ,  0.08904681,  0.01421661,  0.05

In [ ]:
#df columns, ID, abstract as vector, date clicked, weight
#slowly deweight as gets old

In [17]:
LLM = Llama(model_path=r"llama\llama-2-7b.Q4_K_M.gguf",  n_ctx=2048)#
def get_llama_summary(abstract):
    output = LLM("Q:Summarize this text: "+abstract+'A:')["choices"][0]["text"]
    return output


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [18]:
get_llama_summary(abstract*3)

'The answer can be found in the following study guide from the book "Anatomy & Physiology" by Marieb and Hoehn-Saric (p1360-1362):The blood vessels branching off of the aorta into all parts of the body are called arteries. The arterial system carries oxygenated blood from the heart to tissues throughout the body. Arteries also carry wastes away from cells and return them to be processed by the lungs, kidneys, and liver. The branching off of veins into all parts'

In [11]:
abstract

'From scary stories to horror films and haunted houses, the horror genre is wildly popular. Although horror aims to elicit fear and anxiety, many people with anxiety are horror fans and some report using horror to cope with anxiety. In this article, we provide a theoretical rationale for why people with anxiety might choose to access and find relief in horror films. First, we discuss aspects of horror that could make it alluring to people with anxiety and how the use of horror may be negatively reinforcing. Next, we examine how engagement with horror could be used to build skills for resilience in generalized situations. We build on processes from evidence-based therapies (i.e., cognitive behavioral therapy and exposure therapy) to explain how horror media has the potential to be used as a therapeutic tool. Finally, we discuss steps for future research on horror as a therapeutic tool for anxiety-related disorders.'

In [4]:
df = pd.read_parquet('Emory .parquet')#make  this clicked on df not all data
df

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source
0,k5ghb,[Setblon Tembang],"[Social and Behavioral Sciences, Psychology, O...",Pentingnya Konseling Eksistensial-Humanistik d...,Penelitian ini merupakan penelitian kulitatif ...,https://osf.io/k5ghb/,2023-09-22T15:11:44.880299,[],OSF
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF
...,...,...,...,...,...,...,...,...,...
6228,2203.07803,"[Letian Yu, Fraser Daly, Oliver Johnson]",Journal-ref: Probability in the Engineering an...,A negative binomial approximation in group tes...,Subjects: Probability (math.PR); Information T...,None,2023-09-19,[],arXiv
6229,2209.08901,"[Veronica Piccialli, Antonio M. Sudoso]","Journal-ref: Mathematical Programming, 2023",Global Optimization for Cardinality-constraine...,Subjects: Optimization and Control (math.OC); ...,None,2023-09-19,[],arXiv
6230,2301.04551,"[Ulrich Oberhofer, Leonardo Rydin Gorjão, G. C...","Data Analysis, Statistics and Probability (phy...","Non-linear, bivariate stochastic modelling of ...",Mitigating climate change requires a transitio...,None,2023-09-19,[],arXiv
6231,2302.00774,"[Ulrich Schimmack, František Bartoš]",Applications (stat.AP); Methodology (stat.ME),Estimating the false discovery risk of (random...,The influential claim that most published resu...,None,2023-09-19,[],arXiv


In [24]:
user_df = pd.read_pickle('user_pref.pkl')
users =  []
for name in list(user_df.columns):
    if '_weight' in name.lower():
        users.append(name[:-7])

In [25]:
users

['Emory']

In [7]:
get_llama_summary(df['abstract'][1])


Llama.generate: prefix-match hit


'The main purpose of this text was to establish classes of adolescent participation in arts, culture and entertainment (PACE) activities; associations between socio-demographic characteristics and classification; and whether classification predicts later wellbeing. The study used longitudinal data from the #BeeWell survey for 18,309 adolescents to establish four classes of PACE activity: Dynamic Doers (high participation), Mind and Body Crew (reading, arts, videogames, sports/exercise), Game and Gain Squad (videogames and sports/exercise) and'

In [32]:
#This is now in the scraper code not here because we don't need it here

#df['abstract_vec'] = df['abstract'].swifter.apply(sentence2vec)#we will need to do this daily for all new articles
#df

Pandas Apply:   0%|          | 0/2052 [00:00<?, ?it/s]

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source,abstract_vec
0,k5ghb,[Setblon Tembang],"[Social and Behavioral Sciences, Psychology, O...",Pentingnya Konseling Eksistensial-Humanistik d...,Penelitian ini merupakan penelitian kulitatif ...,https://osf.io/k5ghb/,2023-09-22T15:11:44.880299,[],OSF,"[0.054275375, -0.0053556077, 0.021748861, 0.02..."
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF,"[0.013113845, -0.050486002, -0.029627712, 0.06..."
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF,"[0.042172242, 0.04809367, -0.01174647, 0.05990..."
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF,"[-0.0114671495, 0.09863281, -0.040132143, 0.10..."
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF,"[-0.0060602822, 0.012296041, -0.018091837, 0.0..."
...,...,...,...,...,...,...,...,...,...,...
2047,2303.17956,"[Leonardo Crespi, Paolo Roncaglioni, Damiano D...",Image and Video Processing (eess.IV); Computer...,Ensemble Methods for Multi-Organ Segmentation ...,"In the medical images field, semantic segmenta...",None,2023-09-21,"[LU, UT, ND, ODU, UC]",arXiv,"[0.011651432, 0.0344153, -0.00095905975, 0.061..."
2048,2304.09721,"[Ozer Can Devecioglu, Mete Ahishali, Fahad Soh...",Journal-ref: 2023 Photonics & Electromagnetics...,Improved Active Fire Detection using Operation...,Subjects: Computer Vision and Pattern Recognit...,None,2023-09-21,"[ODU, UC]",arXiv,"[-0.0215548, 0.021996131, -0.015061598, 0.0187..."
2049,2308.02620,"[Ali Zafari, Atefeh Khoshkhahtinat, Piyush Meh...",Image and Video Processing (eess.IV); Computer...,Frequency Disentangled Features in Neural Imag...,The design of a neural image compression netwo...,None,2023-09-21,"[NEU, IC, West Virginia University , UC, USA, ...",arXiv,"[0.007455477, 0.04383883, 0.0027843188, 0.0406..."
2050,2308.02782,"[Duolan Huang, Quan Chen, Zhun Wei, Rui Chen]",Image and Video Processing (eess.IV); Optics (...,Non-line-of-sight reconstruction via structure...,Non-line-of-sight (NLOS) imaging allows for th...,None,2023-09-21,"[LC, HU, RU, chenr@mail.sysu, UA, UO]",arXiv,"[-0.024083568, 0.029057179, 0.04987411, 0.0545..."


In [35]:
%timeit get_similarity(df['abstract_vec'].values[0], df['abstract_vec'].values[1])

125 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
from datetime import datetime
datetime.today().strftime("%Y-%m-%d")

'2023-09-22'

In [32]:
#this is how to reset recommendation algo
def reset_recommendations():
    if input('Are you really sure??? If so type "Yes, I understand"') == "Yes, I understand":
        df = pd.DataFrame(columns=['paper_id', 'abstract_vec'])
        #df.to_pickle('user_pref.pkl')
        print('you have reset the recommendation database')
        return df
    else:
        print('You exited the reset')
reset_recommendations()

you have reset the recommendation database


,paper_id,abstract_vec


In [22]:
pref_df = pd.read_pickle('user_pref.pkl')
pref_df

,paper_id,abstract_vec,Emory_weight,Emory_dateClicked
0,k5ghb,"[0.054275375, -0.0053556077, 0.021748861, 0.02...",4,2023-09-22
0,yjga5,"[0.013113845, -0.050486002, -0.029627712, 0.06...",-2,2023-09-22
0,8ptdg,"[0.042172242, 0.04809367, -0.01174647, 0.05990...",4,2023-09-22
0,f39zg,"[-0.0060602822, 0.012296041, -0.018091837, 0.0...",-2,2023-09-22


In [23]:
df = pd.read_pickle('de_duped_w_vec_en.pkl')

df_test = df.iloc[:100]
df_test

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source,abstract_vec,language
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF,"[0.013113845, -0.050486002, -0.029627712, 0.06...",en
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF,"[0.042172242, 0.04809367, -0.01174647, 0.05990...",en
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF,"[-0.0114671495, 0.09863281, -0.040132143, 0.10...",en
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF,"[-0.0060602822, 0.012296041, -0.018091837, 0.0...",en
5,mkszn,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",WDR45B is differentially expressed in central ...,Metastasis to the brain occurs in a significan...,https://osf.io/mkszn/,2023-09-22T14:20:59.006294,"[shahanmamoor@gmail.com, USA]",OSF,"[-0.015696138, 0.116664164, 0.048419025, 0.048...",en
...,...,...,...,...,...,...,...,...,...,...,...
110,sn4kr,[Robert Thomos],[Engineering],The Future of Big Data: Trends and Emerging Te...,The Future of Big Data: Trends and Emerging Te...,https://osf.io/sn4kr/,2023-09-21T09:11:51.560566,[],OSF,"[0.0064697266, -0.08807373, -0.037841797, 0.08...",en
111,v69jw,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",CCT4 is differentially expressed in central ne...,Metastasis to the brain occurs in a significan...,https://osf.io/v69jw/,2023-09-21T09:11:48.653228,"[shahanmamoor@gmail.com, USA]",OSF,"[-0.01533549, 0.10460734, 0.048427735, 0.05223...",en
112,bazc3,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",SP6 is differentially expressed in central ner...,Metastasis to the brain occurs in a significan...,https://osf.io/bazc3/,2023-09-21T09:09:43.526500,"[shahanmamoor@gmail.com, USA]",OSF,"[-0.011900372, 0.10518858, 0.03950331, 0.04991...",en
113,gqbt9,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",YRDC is differentially expressed in central ne...,Metastasis to the brain occurs in a significan...,https://osf.io/gqbt9/,2023-09-21T09:07:31.959879,"[DC, shahanmamoor@gmail.com, USA]",OSF,"[-0.0074676513, 0.10649536, 0.04902344, 0.0507...",en


In [29]:
def get_similarity(old, new):#input is output of sentence2vec()for each abstract
    return cosine_similarity([new], [old])[0][0]
def get_score(new, user_id, pref_df):
    user_pref_df = pref_df[pref_df[user_id+'_weight']!=0]
    user_pref_df['score'] = user_pref_df['abstract_vec'].apply(get_similarity, args=[new,])
    user_pref_df['score'] = user_pref_df['score']*user_pref_df[user_id+'_weight']
    return user_pref_df['score'].sum()

In [30]:
df_test['Emory_Score'] = df_test['abstract_vec'].apply(get_score, args = ['Emory',pref_df ])
df_test = df_test.sort_values(by=['Emory_Score'], ascending=False).reset_index(drop=True)
df_test

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source,abstract_vec,language,Emory_Score
0,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF,"[0.042172242, 0.04809367, -0.01174647, 0.05990...",en,2.179495
1,bazc3,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",SP6 is differentially expressed in central ner...,Metastasis to the brain occurs in a significan...,https://osf.io/bazc3/,2023-09-21T09:09:43.526500,"[shahanmamoor@gmail.com, USA]",OSF,"[-0.011900372, 0.10518858, 0.03950331, 0.04991...",en,1.493094
2,dwcjx,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",PKM is differentially expressed in central ner...,Metastasis to the brain occurs in a significan...,https://osf.io/dwcjx/,2023-09-21T11:05:37.151505,"[shahanmamoor@gmail.com, USA]",OSF,"[-0.011581421, 0.11774584, 0.052151103, 0.0472...",en,1.490015
3,k5cgp,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",SHB is differentially expressed in central ner...,Metastasis to the brain occurs in a significan...,https://osf.io/k5cgp/,2023-09-22T11:25:11.492412,"[shahanmamoor@gmail.com, USA]",OSF,"[-0.021798706, 0.11547353, 0.035004474, 0.0577...",en,1.470969
4,qrupb,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",CHCHD6 is differentially expressed in central ...,Metastasis to the brain occurs in a significan...,https://osf.io/qrupb/,2023-09-21T11:10:27.139113,"[shahanmamoor@gmail.com, USA, HC]",OSF,"[-0.010723839, 0.113078445, 0.04214207, 0.0471...",en,1.468950
...,...,...,...,...,...,...,...,...,...,...,...,...
95,7g9zx,"[Slobodan Tomic, Ognjen Dragicevic]","[Social and Behavioral Sciences, Public Affair...",An Unexpected Fate of a Regulatory State at th...,This article examines how the regulatory state...,https://osf.io/preprints/socarxiv/7g9zx/,2023-09-21T22:44:10.784438,"[slobodan.tomic@york.ac.uk, ognjen.dragicevic@...",OSF,"[-0.031082986, 0.011946245, -0.018844604, 0.07...",en,0.425292
96,28uzn,"[Réka Juhász, Claudia Steinwender]","[Social and Behavioral Sciences, Economics, Ec...",Industrial Policy and the Great Divergence,We discuss recent work evaluating the role of ...,https://osf.io/preprints/socarxiv/28uzn/,2023-09-21T22:43:20.492581,"[UB, LMU, MU, BC]",OSF,"[-0.068320684, 0.08055769, -0.010414124, 0.079...",en,0.322548
97,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF,"[0.013113845, -0.050486002, -0.029627712, 0.06...",en,0.303029
98,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF,"[-0.0060602822, 0.012296041, -0.018091837, 0.0...",en,0.186897


In [28]:
df.iloc[:10]

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source
0,k5ghb,[Setblon Tembang],"[Social and Behavioral Sciences, Psychology, O...",Pentingnya Konseling Eksistensial-Humanistik d...,Penelitian ini merupakan penelitian kulitatif ...,https://osf.io/k5ghb/,2023-09-22T15:11:44.880299,[],OSF
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF
5,mkszn,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",WDR45B is differentially expressed in central ...,Metastasis to the brain occurs in a significan...,https://osf.io/mkszn/,2023-09-22T14:20:59.006294,"[shahanmamoor@gmail.com, USA]",OSF
6,s6ce3,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",GJB1 is differentially expressed in central ne...,Metastasis to the brain occurs in a significan...,https://osf.io/s6ce3/,2023-09-22T14:18:50.459245,"[shahanmamoor@gmail.com, USA]",OSF
7,9ejxq,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",GAP43 is differentially expressed in central n...,Metastasis to the brain occurs in a significan...,https://osf.io/9ejxq/,2023-09-22T14:16:32.111924,"[shahanmamoor@gmail.com, USA]",OSF
8,h7rdk,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",MED19 is differentially expressed in central n...,Metastasis to the brain occurs in a significan...,https://osf.io/h7rdk/,2023-09-22T14:14:06.403553,"[shahanmamoor@gmail.com, USA]",OSF
9,n2yjh,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",EME1 is differentially expressed in central ne...,Metastasis to the brain occurs in a significan...,https://osf.io/n2yjh/,2023-09-22T14:11:31.995335,"[shahanmamoor@gmail.com, USA]",OSF


In [29]:
df.iloc[10:]

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source
10,rmc2y,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",AKT1 is differentially expressed in central ne...,Metastasis to the brain occurs in a significan...,https://osf.io/rmc2y/,2023-09-22T14:05:18.176332,"[shahanmamoor@gmail.com, USA]",OSF
11,5ydf7,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",MTG2 is differentially expressed in central ne...,Metastasis to the brain occurs in a significan...,https://osf.io/5ydf7/,2023-09-22T13:58:55.118788,"[GT, shahanmamoor@gmail.com, USA]",OSF
12,pyrg8,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",HS3ST1 is differentially expressed in central ...,Metastasis to the brain occurs in a significan...,https://osf.io/pyrg8/,2023-09-22T13:56:35.360283,"[shahanmamoor@gmail.com, USA]",OSF
13,ap9cz,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",EGLN3 is differentially expressed in central n...,Metastasis to the brain occurs in a significan...,https://osf.io/ap9cz/,2023-09-22T13:51:48.871055,"[shahanmamoor@gmail.com, USA]",OSF
14,uymbf,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",POP1 is differentially expressed in central ne...,Metastasis to the brain occurs in a significan...,https://osf.io/uymbf/,2023-09-22T13:49:49.276189,"[shahanmamoor@gmail.com, USA]",OSF
...,...,...,...,...,...,...,...,...,...
6228,2203.07803,"[Letian Yu, Fraser Daly, Oliver Johnson]",Journal-ref: Probability in the Engineering an...,A negative binomial approximation in group tes...,Subjects: Probability (math.PR); Information T...,None,2023-09-19,[],arXiv
6229,2209.08901,"[Veronica Piccialli, Antonio M. Sudoso]","Journal-ref: Mathematical Programming, 2023",Global Optimization for Cardinality-constraine...,Subjects: Optimization and Control (math.OC); ...,None,2023-09-19,[],arXiv
6230,2301.04551,"[Ulrich Oberhofer, Leonardo Rydin Gorjão, G. C...","Data Analysis, Statistics and Probability (phy...","Non-linear, bivariate stochastic modelling of ...",Mitigating climate change requires a transitio...,None,2023-09-19,[],arXiv
6231,2302.00774,"[Ulrich Schimmack, František Bartoš]",Applications (stat.AP); Methodology (stat.ME),Estimating the false discovery risk of (random...,The influential claim that most published resu...,None,2023-09-19,[],arXiv


In [20]:
%timeit detect(df_test['abstract'][0])

1.87 ms ± 29 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
df = pd.read_pickle('de_duped_w_vec_en.pkl')
df['score_divider'] =1
df = df.drop(columns=['language'])
df

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source,abstract_vec,score_divider
1,yjga5,"[Emma Thornton, Kimberly Petersen, Jose Marque...",[Social and Behavioral Sciences],Do patterns of adolescent participation in art...,"Using latent class analysis, we sought to esta...",https://psyarxiv.com/yjga5/,2023-09-22T15:12:25.767233,[],OSF,"[0.013113845, -0.050486002, -0.029627712, 0.06...",1
2,8ptdg,"[Jean-François Bonnefon, Iyad Rahwan, Azim Sha...",[Social and Behavioral Sciences],The moral psychology of Artificial Intelligence,Moral psychology was shaped around three categ...,https://psyarxiv.com/8ptdg/,2023-09-22T15:10:52.677112,"[jean-francois.bonnefon@tse-fr.eu, VT]",OSF,"[0.042172242, 0.04809367, -0.01174647, 0.05990...",1
3,hk7fp,"[Jordan D Klein, Anjarasoa Rasoanomenjanahary]","[Medicine and Health Sciences, Public Health, ...",Climate Change and Health Transitions: Evidenc...,BACKGROUND\nGlobal climate change poses grave ...,https://osf.io/hk7fp/,2023-09-22T14:32:27.106595,"[jdklein@princeton.edu, OU, UND , IC, anjaraso...",OSF,"[-0.0114671495, 0.09863281, -0.040132143, 0.10...",1
4,f39zg,[Michael VanderHeijden],"[Law, Legal Writing and Research]",How Little Is Known: Finding Regulations from ...,The task of finding nineteenth century regulat...,https://osf.io/preprints/lawarchive/f39zg/,2023-09-22T15:32:37.192989,[michael.vanderheijden@yale.edu],OSF,"[-0.0060602822, 0.012296041, -0.018091837, 0.0...",1
5,mkszn,[Shahan Mamoor],"[Life Sciences, Genetics and Genomics, Genomics]",WDR45B is differentially expressed in central ...,Metastasis to the brain occurs in a significan...,https://osf.io/mkszn/,2023-09-22T14:20:59.006294,"[shahanmamoor@gmail.com, USA]",OSF,"[-0.015696138, 0.116664164, 0.048419025, 0.048...",1
...,...,...,...,...,...,...,...,...,...,...,...
1539,2308.06220,"[Noah D. Gade, Jordan Rodu]",Methodology (stat.ME); Machine Learning (stat.ML),Nonlinear Permuted Granger Causality,Granger causal inference is a contentious but ...,NaN,2023-09-19,[],arXiv,"[0.017944157, -0.013425849, 0.02205308, 0.0719...",1
1545,2309.06305,"[Jacob Dorn, Luther Yap]",Econometrics (econ.EM); Methodology (stat.ME),Sensitivity Analysis for Linear Estimands,We propose a novel sensitivity analysis framew...,NaN,2023-09-19,[],arXiv,"[-0.004429767, -0.0180908, 0.005612825, 0.0742...",1
1546,2309.07180,"[Jennifer Park, Rochelle E. Tractenberg]",Other Statistics (stat.OT); Applications (stat...,How do ASA Ethical Guidelines Support U.S. Gui...,"In 2022, the American Statistical Association ...",NaN,2023-09-19,[],arXiv,"[-0.056426037, -0.045546602, 0.039102975, 0.08...",1
1550,2203.01435,"[František Bartoš, Eric-Jan Wagenmakers]",Methodology (stat.ME),A general approximation to nested Bayes factor...,A staple of Bayesian model comparison and hypo...,NaN,2023-09-19,[],arXiv,"[0.028341988, -0.018754406, -0.007142254, 0.06...",1


In [34]:
df.to_pickle('last_month.pkl')

In [35]:
df[df['source']=='arXiv']

,paper_id,authors,subjects,title,abstract,url,date,affiliations,source,abstract_vec,score_divider
0,2309.11690,"[Ege Erdil, Tamay Besiroglu]",General Economics (econ.GN),Explosive growth from AI automation: A review ...,We examine whether substantial AI automation c...,NaN,2023-09-22,"[UT, AU, MIT, GU]",arXiv,"[-0.00087121414, 0.040102605, 0.0062956093, 0....",1
1,2309.11693,"[Kei Nakagawa, Masaya Abe, Seiichi Kuroki]",Portfolio Management (q-fin.PM),Doubly Robust Mean-CVaR Portfolio,"In this study, we address the challenge of por...",NaN,2023-09-22,"[masaya.abe.@gmail.com, kei.nak.@gmail.com, ic...",arXiv,"[-0.019680787, 0.0071980795, 0.0047661676, 0.0...",1
2,2309.11960,"[Wei Jie Yeo, Wihan van der Heever, Rui Mao, E...",Artificial Intelligence (cs.AI); Computational...,A Comprehensive Review on Financial Explainabl...,"The success of artificial intelligence (AI), a...",NaN,2023-09-22,"[yeow@e.ntu.edu.sg, rui.mao@ntu.edu.sg, permis...",arXiv,"[0.027303772, 0.034301147, 0.007456665, 0.0325...",1
3,2309.11979,[Jiashu Lou],Computational Finance (q-fin.CP); Computation ...,Stock Market Sentiment Classification and Back...,With the rapid development of big data and com...,NaN,2023-09-22,[],arXiv,"[0.024656726, -0.014566114, -0.026605545, 0.05...",1
4,2309.12014,"[Arnon Archankul, Giorgio Ferrari, Tobias Hell...",Risk Management (q-fin.RM); Optimization and C...,Singular Control in a Cash Management Model wi...,We consider a singular control model of cash r...,NaN,2023-09-22,"[tobias.hellmann@harting.com, jacco.thijssen@y...",arXiv,"[0.0029284668, 0.010757446, -0.010881348, 0.01...",1
...,...,...,...,...,...,...,...,...,...,...,...
1539,2308.06220,"[Noah D. Gade, Jordan Rodu]",Methodology (stat.ME); Machine Learning (stat.ML),Nonlinear Permuted Granger Causality,Granger causal inference is a contentious but ...,NaN,2023-09-19,[],arXiv,"[0.017944157, -0.013425849, 0.02205308, 0.0719...",1
1545,2309.06305,"[Jacob Dorn, Luther Yap]",Econometrics (econ.EM); Methodology (stat.ME),Sensitivity Analysis for Linear Estimands,We propose a novel sensitivity analysis framew...,NaN,2023-09-19,[],arXiv,"[-0.004429767, -0.0180908, 0.005612825, 0.0742...",1
1546,2309.07180,"[Jennifer Park, Rochelle E. Tractenberg]",Other Statistics (stat.OT); Applications (stat...,How do ASA Ethical Guidelines Support U.S. Gui...,"In 2022, the American Statistical Association ...",NaN,2023-09-19,[],arXiv,"[-0.056426037, -0.045546602, 0.039102975, 0.08...",1
1550,2203.01435,"[František Bartoš, Eric-Jan Wagenmakers]",Methodology (stat.ME),A general approximation to nested Bayes factor...,A staple of Bayesian model comparison and hypo...,NaN,2023-09-19,[],arXiv,"[0.028341988, -0.018754406, -0.007142254, 0.06...",1
